In [1]:
import tensorflow as tf
import numpy as np
from matplotlib import pyplot as plt
import datetime
import sys
sys.path.append("..")
from train import BodyPartsMeasurementTrainer
from body_parts_measurement_data_generator import BodyPartsMeasurementDataGenerator
import tensorflow as tf
from evaluator import Evaluator
from model.model import get_model
from tqdm import tqdm

In [2]:
config = {
    # model
    'input_shape': [256,256,3],     # 훈련 이미지 크기
    'batch_size': 4,                # 배치 사이즈
    'path_pretrained': None,        # pretrained 모델 경로
    'type_backbone': "blazepose",   # backbone type (blazepose, mobilenet_v3)
    
    # loss
    'type_loss_fn': 'wing',         # 손실 함수 설정 (wing, mae)
    
    # data
    'seg_shape': [64,64],           # segmentation 크기 *미사용
    'path_classes': "../seg_classes.txt",   # segmentation class 정보 *미사용
    'shuffle': True,                # 데이터 섞기
    'is_normalized': False,         # normalize 데이터
    'is_with_seg': False,           # segmentation 사용 여부 *미사용
    'path_dataset': "D:\\data\\body_parts_measurement", # 데이터 경로
    ## attention type              
    'type_attention': "regression", # attention 종류 (regression, categorical, none)
    'num_category_bmi': 10,         # categorical 시의 bmi category 갯수 변수
    'num_category_height': 10,      # categorical 시의 height category 갯수 변수

    # train
    'epochs': 30,                   # 훈련 epoch 수
    'eval_term': 1                  # 평가 빈도
}

In [3]:
model = get_model(config)

In [4]:
# model.load_weights('blazepose_attention_31features_731train_mf/blazepose_attention_0_2.994556342230903.h5')
model.load_weights('blazepose_attention_31features_1002train_mf/blazepose_attention_0_3.2034787193590604.h5')

In [4]:
data_generator_test = BodyPartsMeasurementDataGenerator(config, data_type="test")
len(data_generator_test)

list_user_dirs ['F004', 'F022', 'F027', 'F037', 'F057', 'F060', 'F061', 'F063', 'F064', 'F069', 'F088', 'F089', 'F098', 'F109', 'F114', 'F121', 'F129', 'F134', 'F170', 'F188', 'F192', 'F206', 'F214', 'F217', 'F224', 'F226', 'F239', 'F258', 'F259', 'F308', 'F312', 'F314', 'F324', 'F325', 'F327', 'F329', 'F332', 'F333', 'F344', 'F352', 'F356', 'F364', 'F379', 'F380', 'F383', 'F413', 'F419', 'F423', 'F426', 'F442', 'F492', 'F493', 'M009', 'M022', 'M024', 'M027', 'M041', 'M056', 'M065', 'M072', 'M074', 'M081', 'M088', 'M090', 'M101', 'M113', 'M125', 'M126', 'M128', 'M139', 'M153', 'M167', 'M172', 'M173', 'M175', 'M195', 'M199', 'M221', 'M225', 'M240', 'M255', 'M260', 'M301', 'M306', 'M308', 'M310', 'M315', 'M336', 'M338', 'M345', 'M353', 'M362', 'M365', 'M372', 'M384', 'M401', 'M441', 'M450', 'M462', 'M464', 'M465', 'M466', 'M473', 'M485', 'M491']


30240

In [6]:
evaluator = Evaluator(data_generator_test, model)

In [7]:
evaluator.run_evaluation()

 82%|████████▏ | 24681/30240 [2:13:57<27:52,  3.32it/s]  

error OpenCV(4.5.3) C:\Users\runneradmin\AppData\Local\Temp\pip-req-build-c2l3r8zm\opencv\modules\imgproc\src\resize.cpp:4051: error: (-215:Assertion failed) !ssize.empty() in function 'cv::resize'



100%|██████████| 30240/30240 [2:44:44<00:00,  3.06it/s]


In [8]:
np_diff_parts_measurements = np.squeeze(np.array(evaluator.list_diff_parts_measurements))
np_diff_percentage_parts_measurements = np.squeeze(np.array(evaluator.list_diff_percentage_parts_measurements))
list_part_mapes = []
for idx in range(np_diff_parts_measurements.shape[1]) :
    part_mape = sum(np_diff_percentage_parts_measurements[:,idx])/np_diff_parts_measurements.shape[0]
    print(idx, part_mape)
    #print(sum(np_diff_percentage_parts_measurements[:,idx])/np_diff_parts_measurements.shape[0])
    list_part_mapes.append(part_mape)
print("mape",sum(list_part_mapes)/len(list_part_mapes))

0 0.12476265917276143
1 0.041094798135945965
2 0.061170808992874554
3 0.11555510882491876
4 0.13487792100704843
5 0.04244377068651675
6 0.24284584491640093
7 0.03797059532622887
8 0.03504382386216877
9 0.1750653760514688
10 0.083661787484856
11 0.07641139240750593
12 0.06533858123513538
13 0.19006077327604928
14 0.0453961984237471
15 0.04583019170669868
16 0.07960025961348301
17 0.035635785065126435
18 0.034620783848326134
19 0.04251038172498105
20 0.03571259517770826
21 0.04167059543366567
22 0.03893934871177923
23 0.08076866443680712
24 0.04811084598017413
25 0.033569452438636245
26 0.05361421186706462
27 0.05744858358475839
28 0.030285196556262016
29 0.03638407207185011
30 0.04266762423082188
mape 0.0712602591048958


In [ ]:
iter_data_generator_test = iter(data_generator_test)

In [ ]:
data = next(iter_data_generator_test)
print(data)

In [ ]:
plt.imshow(data[0][0,:,:,::-1].astype(np.int))
plt.show()

In [ ]:
result = model.predict([data[0], data[2]])

In [ ]:
result[0]

In [ ]:
data[1][0]

In [12]:
config = {
        # model
        'input_shape': [256,256,3],
        'batch_size': 1,
        'path_pretrained': None, 
        'type_backbone': "blazepose", 
        
        # loss
        'type_loss_fn': 'wing',
        
        # data
        'seg_shape': [64,64],
        'path_classes': "../seg_classes.txt",
        'shuffle': True, 
        'is_normalized': False, 
        'is_with_seg': False, 
        'path_dataset': "D:\\data\\body_parts_measurement",
        ## attention type
        'type_attention': "regression", #regression, categorical, none
        'num_category_bmi': 10,
        'num_category_height': 10,
        'has_filename': True,

        # train
        'epochs': 30,
        'eval_term': 1
    }
model = get_model(config)
model.summary()
model.load_weights('blazepose_attention_31features_1002train_mf/blazepose_attention_0_3.2034787193590604.h5')
data_generator_test = BodyPartsMeasurementDataGenerator(config, data_type="test")

Model: "model_5"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_9 (InputLayer)            [(1, 256, 256, 3)]   0                                            
__________________________________________________________________________________________________
conv2d_118 (Conv2D)             (1, 128, 128, 24)    672         input_9[0][0]                    
__________________________________________________________________________________________________
sequential_144 (Sequential)     (1, 128, 128, 24)    840         conv2d_118[0][0]                 
__________________________________________________________________________________________________
tf.__operators__.add_18 (TFOpLa (1, 128, 128, 24)    0           conv2d_118[0][0]                 
                                                                 sequential_144[0][0]       

In [13]:
f_w = open("eval_results_test.txt","w")
f_w.write("Evaluation Started At "+datetime.datetime.now().strftime('%Y-%m-%d %H:%M:%S') + "\n")

list_mape = []
list_diff_percentage_parts_measurements = []



for idx in tqdm(range(len(data_generator_test))) :
    try :
        batch_data = data_generator_test.get_data(idx)
    except :
        print("idx",idx,"wrong")
    batch_images, batch_body_parts_measurement, batch_categorical_bmi_and_height, filename = batch_data
    batch_images, batch_body_parts_measurement, batch_categorical_bmi_and_height = np.expand_dims(batch_images, axis=0), np.expand_dims(batch_body_parts_measurement, axis=0), np.expand_dims(batch_categorical_bmi_and_height, axis=0)

    pd_parts_measurements = model.predict([batch_images, batch_categorical_bmi_and_height])
    batch_diff_parts_measurements = abs(batch_body_parts_measurement - pd_parts_measurements)
    batch_diff_percentage_parts_measurements = abs(batch_diff_parts_measurements / batch_body_parts_measurement)
    

    mape = sum(batch_diff_percentage_parts_measurements[0]) / len(batch_diff_percentage_parts_measurements[0])

    str_filename = str(idx) + " filename: " + filename + "\n"
    str_gt = "gt: " + ", ".join(str(x) for x in batch_body_parts_measurement[0]) + "\n"
    str_pd = "pd: " + ", ".join(str(x) for x in pd_parts_measurements[0]) + "\n"
    str_pe = "ape: " + ", ".join(str(x) for x in batch_diff_percentage_parts_measurements[0]) + "\n"
    str_mape = "MAPE : " + str(mape) + "\n\n"

    f_w.write(str_filename)
    f_w.write(str_gt)
    f_w.write(str_pd)
    f_w.write(str_pe)
    f_w.write(str_mape)

    list_mape.append(mape)
    list_diff_percentage_parts_measurements.append(batch_diff_percentage_parts_measurements)

np_diff_percentage_parts_measurements = np.squeeze(np.array(list_diff_percentage_parts_measurements))
for idx in range(np_diff_percentage_parts_measurements.shape[1]) :
    part_mape = sum(np_diff_percentage_parts_measurements[:,idx])/np_diff_percentage_parts_measurements.shape[0]
    f_w.write("Part MAPE "+ str(idx) + ": " + str(part_mape) + "\n")

f_w.write("MAPE for Test Dataset : " + str(sum(list_mape)/len(list_mape)) + "\n")
f_w.write("Evaluation Ended At "+datetime.datetime.now().strftime('%Y-%m-%d %H:%M:%S') + "\n")
f_w.close()

 17%|█▋        | 20601/120960 [33:07<2:26:53, 11.39it/s]

idx 20600 wrong


100%|██████████| 120960/120960 [3:19:16<00:00, 10.12it/s] 
